# VAR模型

## 原理讲解

可以参考公众号：[Stata: VAR (向量自回归) 模型](https://mp.weixin.qq.com/s/OxG-sk1MJUB8HV9V6y6Byw)

## matlab 实现

### VAR 模型

VAR.m
```matlab
function [beta,resid,cov_mat,AIC] = VAR(Y,X,p)
% 注意:只是实现了两变量的VAR模型，思考如何实现任意个变量的VAR
% p - 滞后阶数
% AIC - 信息准则

% 1.估计系数
[ADLy,ADLx] = ADLxx(Y,X,p,p);
[beta1,~,resid1] = regress(ADLy,ADLx);

[ADLy,ADLx] = ADLxx(X,Y,p,p);
[beta2,~,resid2] = regress(ADLy,ADLx);

beta = [beta1';beta2'];
resid = [resid1,resid2];

% 2.估计AIC值(eviews调整自由度之后的)
T = length(ADLy);
cov_mat = resid'*resid/(T-3);
AIC = log(det(cov_mat))+2*(2*p+1)*2/T;

% % 1.载入数据
% data = xlsread('C:\Users\Administrator\Desktop\hourse.xlsx');
% f1 = data(:,2); f2 = data(:,3); e = data(:,6);
% 
% % 2.估计VAR模型
% [beta,resid,AIC] = VAR(f1,e,2);
```

### 脉冲响应函数

OIRF1.m
```matlab
function [beta,IR] = OIRF1(Y,X,num,IMP)

% num - 脉冲期数

% 1.估计VAR模型参数
[beta,~,cov_mat] = VAR(Y,X,1);

% 2.脉冲响应函数
% 2.1 正交化分解，估计P矩阵
P = chol(cov_mat, 'lower');
% 2.2 估计IR,s=1期为ADt,s=k，则为A^(k)Dt
b = beta(:,2:3);
SHOCK = zeros(2,1);
if IMP == 1
    SHOCK(1,1) = 1; 
elseif IMP == 2
    SHOCK(2,1) = 1; 
end
IR = zeros(num,2);
IR(1,:) = b*(P*SHOCK);
for s=2:num, IR(s,:) = (b*IR(s-1,:)')'; end
```